### Rasna's Clipping File

This code reads annotations from a file video_annot.txt, opens each video file, segments it into 3-second clips, and writes each clip to a separate video file in the Video_Clips directory. It also creates an annotation file vid_annotation.txt that contains the path to each clip and a label (1 or 0) indicating whether the clip contains a seizure event or not, based on the time range provided in the annotations.

For some reason, this may write the path with `_ '` rather than just `_` and uses `\` instead of `/`. Find and replace in the annotation file for each of these errors. You will have to change the video names accordingly. Do that with the last cell in the notebook.

Finally, if you run into an error where mmmaction can't seem to find the videos, just move them into a double nested folder of the name name (for example, I put mine in `video_clips/video_clips/`)

Takes around an hour to clip our dataset. Move the file created to the annotations file after completion (am not creating there to avoid overwrite on accidental run)

In [2]:
import os
import cv2
from tqdm import tqdm

In [4]:
# Paths

input_dir = "videos"
output_dir = "video_clips"
annotation_file = "test_vid_annotation.txt"
master_file = "annotations/video_annotations.txt"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Open the annotation file for writing
with open(annotation_file, "a+") as f:
    # Read the annotations from the master file
    with open(master_file, "r") as annot_file:
        annotations = [eval(line.strip()) for line in annot_file]

    # Iterate over each video file and its annotations
    for video_path, video_id, patient_id, time_range in tqdm(annotations):
        # Open the video file and get the properties
        video = cv2.VideoCapture(video_path)
        fps = video.get(cv2.CAP_PROP_FPS)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

        # Calculate the number of 3-second segments
        num_segments = int(total_frames // (fps * 3))

        # Initialize the segment counter
        segment_count = 0

        # Iterate over the segments
        for i in range(num_segments):
            # Set the starting frame for the current segment
            start_frame = int(i * fps * 3)

            # Set the video position to the starting frame
            video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

            # Initialize the frames list for the current segment
            segment_frames = []

            # Read frames for the current segment
            for _ in range(int(fps * 3)):
                ret, frame = video.read()
                if ret:
                    segment_frames.append(frame)
                else:
                    break

            # If frames were successfully read for the segment
            if len(segment_frames) > 0:
                # Create the output segment path with patient_id
                segment_name = f"{patient_id}_{video_id}_Seg_{segment_count}.mp4"
                segment_path = os.path.join(output_dir, segment_name)

                # Write the segment frames to the output video file
                out = cv2.VideoWriter(segment_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame.shape[1], frame.shape[0]))
                for frame in segment_frames:
                    out.write(frame)
                out.release()

                # Calculate the segment start and end times in seconds
                segment_start_time = i * 3
                segment_end_time = (i + 1) * 3

                # Check if the current segment overlaps with any seizure time
                is_seizure = segment_start_time >= time_range[0] and segment_end_time <= time_range[1]

                # Write the annotation for the current segment
                annotation = f"{segment_path} {1 if is_seizure else 0}\n"
                f.write(annotation)

                # Increment the segment counter
                segment_count += 1

        # Release the video capture
        video.release()

print("Video segmentation and annotation completed.")

100%|██████████| 49/49 [56:51<00:00, 69.63s/it]

Video segmentation and annotation completed.


### Script to rename all erroneous video clips

In [4]:
dir_path = "C:/Users/u251245/CVEpilepsy/video_clips"

# Set the text to find and replace
find_text = "_ '"
replace_text = "_"

for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    
    if os.path.isfile(file_path):
        # Check if the file name contains the text to find
        if find_text in filename:
            # Construct the new file name
            new_filename = filename.replace(find_text, replace_text)
            new_file_path = os.path.join(dir_path, new_filename)
            
            # Rename the file
            os.rename(file_path, new_file_path)